In [3]:
import torch
from torch_geometric.data import Data



In [4]:
edge_index = torch.tensor([[0, 1, 1, 2],
                           [1, 0, 2, 1]], dtype=torch.long)


In [5]:
edge_index

tensor([[0, 1, 1, 2],
        [1, 0, 2, 1]])

In [6]:
x = torch.tensor([[-1], [0], [1]], dtype=torch.float)



In [7]:
x

tensor([[-1.],
        [ 0.],
        [ 1.]])

In [8]:
data = Data(x=x, edge_index=edge_index)

In [9]:
data

Data(x=[3, 1], edge_index=[2, 4])

In [10]:
Data(edge_index=[2, 4], x=[3, 1])

Data(x=[2], edge_index=[2])